### Notebook to take part of the Space Ship challenge from Kaggle -> predict whether people are "transported"?

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv(r'C:\data\spaceship-titanic\train.csv')
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
test_data = pd.read_csv(r'C:\data\spaceship-titanic\test.csv')
test_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [4]:
# extract vips who survived
vips = train_data.loc[train_data.VIP == True]["Transported"]
rate_vips = sum(vips)/len(vips)

print("% of vips who were transported:", rate_vips)

% of vips who were transported: 0.38190954773869346


In [5]:
# extract vips who survived
vips = train_data.loc[train_data.VIP == False]["Transported"]
rate_vips = sum(vips)/len(vips)

print("% of not vips who were transported:", rate_vips)

% of not vips who were transported: 0.5063321674104451


In [6]:
# extract cryos who survived
cryos = train_data.loc[train_data.CryoSleep == True]["Transported"]
rate_cryos = sum(cryos)/len(cryos)

print("% of cryos who were transported:", rate_cryos)

% of cryos who were transported: 0.8175831412578202


In [7]:
# extract not cryos who survived
n_cryos = train_data.loc[train_data.CryoSleep == False]["Transported"]
rate_n_cryos = sum(n_cryos)/len(n_cryos)

print("% of cryos who were transported:", rate_n_cryos)

% of cryos who were transported: 0.3289207574921861


In [8]:
from sklearn.ensemble import RandomForestClassifier

features = ['VIP', 'CryoSleep']
y = train_data['Transported']

X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index=False)
print('Your submission was successfully saved!')

Your submission was successfully saved!


In [17]:
home_planets = train_data['HomePlanet']
print(home_planets.unique())
# TODO clean only on features (avoid removing data unecessary)
train_data_cleaned = train_data.dropna()
print(len(train_data_cleaned))

encoded_data = train_data_cleaned.copy()
encoded_data_test = test_data.copy()

['Europa' 'Earth' 'Mars' nan]
6606


#### Prepare the Encoding (OneHot) for the categorical values

In [23]:
import helpers
from sklearn import preprocessing
import numpy as np

ohe = preprocessing.OneHotEncoder()

categories = np.array(list(set(train_data_cleaned['HomePlanet'].astype(str).values))).reshape(-1,1)
ohe.fit(categories)

print(encoded_data['Cabin'].unique())

['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']


In [25]:
#import preprocessing module
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Convert target label to numerical Data
le_homeplanet = LabelEncoder()
encoded_data['HomePlanet'] = le_homeplanet.fit_transform(encoded_data['HomePlanet'])
encoded_data['Destination'] = le_homeplanet.fit_transform(encoded_data['Destination'])
encoded_data["CryoSleep"] = encoded_data["CryoSleep"].astype(int)
encoded_data["VIP"] = encoded_data["VIP"].astype(int)

encoded_data_test['HomePlanet'] = le_homeplanet.fit_transform(encoded_data_test['HomePlanet'])
encoded_data_test['Destination'] = le_homeplanet.fit_transform(encoded_data_test['Destination'])
encoded_data_test["CryoSleep"] = encoded_data_test["CryoSleep"].astype(int)
encoded_data_test["VIP"] = encoded_data_test["VIP"].astype(int)

#Separate training features from target
# X = data.drop(['bank_account'], axis=1)
# y = data['bank_account']

# print(y)
encoded_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1,0,B/0/P,2,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,0,0,F/0/S,2,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,1,0,A/0/S,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,1,0,A/0/S,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,0,0,F/1/S,2,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [32]:
features = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
y = encoded_data['Transported']

X = pd.get_dummies(encoded_data[features])
X_test = pd.get_dummies(encoded_data[features])

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(X, y)
# predictions = model.predict(X_test)

# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
# output.to_csv('submission.csv', index=False)
import tensorflow as tf


model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y, epochs=100)

Epoch 1/100


c:\Users\steph\anaconda3\envs\ds_playground\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


207/207 [==============================] - 1s 2ms/step - loss: 0.5378 - accuracy: 0.7681 - precision: 0.7680 - recall: 0.7731
Epoch 2/100
207/207 [==============================] - 0s 2ms/step - loss: 0.4551 - accuracy: 0.7873 - precision: 0.7820 - recall: 0.8010
Epoch 3/100
207/207 [==============================] - 0s 2ms/step - loss: 0.4435 - accuracy: 0.7896 - precision: 0.7751 - recall: 0.8203
Epoch 4/100
207/207 [==============================] - 0s 2ms/step - loss: 0.4421 - accuracy: 0.7879 - precision: 0.7816 - recall: 0.8034
Epoch 5/100
207/207 [==============================] - 0s 2ms/step - loss: 0.4437 - accuracy: 0.7879 - precision: 0.7762 - recall: 0.8133
Epoch 6/100
207/207 [==============================] - 0s 2ms/step - loss: 0.4399 - accuracy: 0.7926 - precision: 0.7702 - recall: 0.8383
Epoch 7/100
207/207 [==============================] - 0s 2ms/step - loss: 0.4441 - accuracy: 0.7953 - precision: 0.7727 - recall: 0.8410
Epoch 8/100
207/207 [=========================

In [41]:
predictions = model.predict(X_test_scaled)
predictions = predictions.reshape(-1)
print(predictions.shape)
print(X_test_scaled.shape)
len(test_data)


# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
# output.to_csv('submission.csv', index=False)
# print('Your submission was successfully saved!')

207/207 [==============================] - 0s 2ms/step
(6606,)
(6606, 10)


4277